In [ ]:
# Version 1. Descriptive Analysis
from google.colab import files
uploaded = files.upload()
import pandas as pd
df = pd.read_csv('Sample - Superstore.csv', encoding='latin1')
df.info()
df.shape
df.isna().sum()
df.describe()

# Chuyển Dtype của Order Date & Ship Date
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])
print(df.info())

# Check outlier Sales, Profit
# Kết quả đối với Sales có tới 5 đơn (> 10000), khả năng cao là đúng dữ liệu --> Giữ nhưng khi visualize sẽ dùng log scale hoặc remove outlier mode nếu biểu đồ lệch
# Kết quả đối với Profit chỉ có 1 đơn (< -5000) --> Loại để tránh ảnh hưởng dữ liệu
print(df[df['Sales'] > 10000])
print(df[df['Profit'] < - 1000])

# Xóa outlier ở Profit
df.drop(df[df['Profit'] < - 1000].index, inplace= True)
print(df.describe())
print("Đơn hàng Profit < -1000 còn lại:", df[df['Profit'] < -1000].shape[0])

# Download data clean
df.to_csv('Sample - Superstore_clean.csv')
from google.colab import files
files.download('Sample - Superstore_clean.csv')

Saving Sample - Superstore.csv to Sample - Superstore (22).csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   ob

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Version 2. Predictive Analysis
from google.colab import files
uploaded = files.upload()
import pandas as pd
df = pd.read_csv('Sample - Superstore_clean.csv', encoding='latin1', parse_dates=['Order Date', 'Ship Date'], index_col=0)
df.shape
df.info()
df.head()

# Chọn biến đầu vào (features) & biến mục tiêu (Profit)
target = 'Profit'
drop_cols = ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Postal Code', 'State', 'City', 'Country', 'Customer Name', 'Customer ID', 'Product ID', 'Product Name', 'Sub-Category','Profit']
x = df.drop(columns= drop_cols, errors='ignore')
y = df[target]
x.columns.tolist()

# Mã hóa biến categorical
## Kiểm tra số lượng biến cho các Cột object
cat_cols = x.select_dtypes(include='object').columns.tolist()
cat_cols
x[cat_cols].nunique().sort_values(ascending=False)
## Onehot Encode
x_encoded = pd.get_dummies(x, columns = cat_cols, drop_first=True)
x_encoded.shape
## Kiểm tra còn NA không (Vì ML không chấp nhận giá trị NaN)
x_encoded.isna().sum()

# Tách Train/Test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test =train_test_split(x_encoded, y, test_size=0.2, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

# Train bằng Linear Regression trước tiên
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
lr_model = LinearRegression()
lr_model.fit(x_train, y_train)
y_pred = lr_model.predict(x_test)
MAE_lr = mean_absolute_error(y_test, y_pred)
RMSE_lr = np.sqrt(mean_squared_error(y_test, y_pred))
r2_lr = r2_score(y_test, y_pred)
print('MAE lr:', MAE_lr)
print('RMSE lr:', RMSE_lr)
print('R2 Score lr:', r2_lr)

# Train tiếp bằng Random Forest
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(x_train, y_train)
y_pred_rf = rf_model.predict(x_test)
MAE_rf = mean_absolute_error(y_test, y_pred_rf)
RMSE_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)
print('MAE rf:', MAE_rf)
print('RMSE rf:', RMSE_rf)
print('R2 Score rf:', r2_rf)
## Xem Feature Importances
feature_importances = pd.Series(rf_model.feature_importances_, index = x_train.columns)
feature_importances = feature_importances.sort_values(ascending=False)
print('\n Top 10 Feature Importances:')
print(feature_importances.head(10))

# Lưu kết quả tĩnh (metrics, feature importances & visualize lên file Power BI đã tạo ở phần Descriptive Analysis trước đó)
metrics = pd.DataFrame({'Model' : ['Linear Regression', 'Random Forest'], 'MAE' : [MAE_lr, MAE_rf], 'RMSE' : [RMSE_lr, RMSE_rf], 'R2 Score' : [r2_lr, r2_rf]})
metrics.to_csv('metrics_ver2.csv', index=False)
fi = feature_importances.reset_index()
fi.columns = ['Feature', 'Importances']
fi.to_csv('feature_importances_ver2.csv', index=False)
from google.colab import files
files.download('metrics_ver2.csv')
files.download('feature_importances_ver2.csv')

Saving Sample - Superstore_clean.csv to Sample - Superstore_clean (13).csv
<class 'pandas.core.frame.DataFrame'>
Index: 9972 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9972 non-null   int64         
 1   Order ID       9972 non-null   object        
 2   Order Date     9972 non-null   datetime64[ns]
 3   Ship Date      9972 non-null   datetime64[ns]
 4   Ship Mode      9972 non-null   object        
 5   Customer ID    9972 non-null   object        
 6   Customer Name  9972 non-null   object        
 7   Segment        9972 non-null   object        
 8   Country        9972 non-null   object        
 9   City           9972 non-null   object        
 10  State          9972 non-null   object        
 11  Postal Code    9972 non-null   int64         
 12  Region         9972 non-null   object        
 13  Product ID     9972 non-null   object        
 14  Ca

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Tuning bằng RandomizedSearchCV
## Khởi tạo lại biến
from google.colab import files
uploaded = files.upload()
import pandas as pd
df = pd.read_csv('Sample - Superstore_clean.csv', encoding='latin1', parse_dates=['Order Date', 'Ship Date'], index_col=0)
target = 'Profit'
drop_cols = ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Postal Code', 'State', 'City', 'Country', 'Customer Name', 'Customer ID', 'Product ID', 'Product Name', 'Sub-Category','Profit']
x = df.drop(columns= drop_cols, errors='ignore')
y = df[target]
x.columns.tolist()
cat_cols = x.select_dtypes(include='object').columns.tolist()
cat_cols
x[cat_cols].nunique().sort_values(ascending=False)
x_encoded = pd.get_dummies(x, columns = cat_cols, drop_first=True)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_encoded, y, test_size= 0.2, random_state=42)
## Khai báo phạm vi tham số
param_dist = {'n_estimators': [200, 400, 600], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'max_features': ['sqrt', 0.7], 'bootstrap': [True], 'max_samples': [None, 0.85]}
## Khởi tạo model gốc - model trắng đang muốn Tuning
from sklearn.ensemble import RandomForestRegressor
rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)
## Tạo RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(estimator = rf_base, param_distributions = param_dist, n_iter = 25, cv = 3, scoring = 'r2', verbose = 1, random_state = 42, n_jobs = -1)
## Chạy fit
random_search.fit(x_train, y_train)
random_search.best_params_
random_search.best_score_
print("Best Params:", random_search.best_params_)
print("Best R2: ", round(random_search.best_score_, 4))
best_model = random_search.best_estimator_
## Đánh giá trên test set
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
y_pred = best_model.predict(x_test)
MAE = mean_absolute_error(y_test, y_pred)
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
R2 = r2_score(y_test, y_pred)
print("MAE:", MAE)
print("RMSE:", RMSE)
print("R2 Score:", R2)
## -> Đã thử tuning bằng RandomizedSearchCV, nhưng kết quả không vượt baseline. Quyết định giữ mô hình gốc để đảm bảo hiệu quả

# Xuất kết quả
import pandas as pd, json, joblib

## Feature Importance
fi_v3 = pd.DataFrame({"Feature": x_train.columns, "Importance": best_model.feature_importances_}).sort_values("Importance", ascending=False)
fi_v3.to_csv('feature_importances_v3.csv', index=False)

## Prediction
pred_v3 = pd.DataFrame({"Actual Profit": y_test, "Predicted Profit": y_pred})
pred_v3.to_csv('predictions_v3.csv', index=False)

## Metrics
metrics_v3 = pd.DataFrame({"Model": ["Random Forest (Tuned)"], "MAE": [MAE], "RMSE": [RMSE], "R2 Score": [R2]})
metrics_v3.to_csv('metrics_v3.csv', index= False)

## Best params + model
with open("best_params_v3.json", "w") as f:
  json.dump(random_search.best_params_, f, indent=2)
joblib.dump(best_model, "rf_tuned_v3.joblib")

# Download
from google.colab import files as gfiles
for fname in ["feature_importances_v3.csv", "predictions_v3.csv", "metrics_v3.csv", "best_params_v3.json", "rf_tuned_v3.joblib"]:
  gfiles.download(fname)

Saving Sample - Superstore_clean.csv to Sample - Superstore_clean (10).csv
Fitting 3 folds for each of 25 candidates, totalling 75 fits
Best Params: {'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_samples': None, 'max_features': 0.7, 'max_depth': None, 'bootstrap': True}
Best R2:  0.6993
MAE: 20.574493418177905
RMSE: 70.14394165579502
R2 Score: 0.8484176775957384


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
# VERSION 4 - TIME-SERIES FORECASTING
from google.colab import files
upload = files.upload()
import pandas as pd
df = pd.read_csv('Sample - Superstore_clean.csv', low_memory= False)

# Basic EDA
df.info()
df.isnull().sum()
df.isna().sum()

# Actions after Basic EDA
## Change dtype of 'Order Date', 'Ship Date' column from object to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'], errors= 'coerce')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], errors= 'coerce')
df.info()
df.isnull().sum()
df.isna().sum()

# Predictive Analysis Setup
## Set trục thời gian: Order Date
df = df.set_index('Order Date')
## Resample by Week
weekly_data = df['Profit'].resample('W').sum()
## Group by Category
weekly_category = df.groupby(['Category']).resample('W')['Profit'].sum().reset_index()
## Train/Test Split
train = weekly_data[:'2016']
test = weekly_data['2017':]

# Modeling
## Naive Forecast
naive_forecast = test.copy()
naive_forecast[:] = train.iloc[-1]
## Exponential Smoothing (Holt-Winters)
from statsmodels.tsa.holtwinters import ExponentialSmoothing
hw_model = ExponentialSmoothing(train, trend= 'add', seasonal= 'add', seasonal_periods= 52).fit()
hw_forecast = hw_model.forecast(len(test))
print(hw_forecast.head())
## SARIMA
import itertools
import warnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
warnings.filterwarnings('ignore')
baseline_model = SARIMAX(train, order=(0,1,0), seasonal_order=(0,1,0,52))
baseline_results = baseline_model.fit(disp=False)
print("Baseline AIC:", baseline_results.aic)
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
P = D = Q = [0, 2]
seasonal_pdq = [(0,1,0,52), (1,1,0,52)]
best_aic = float('inf')
best_param= None
for param in pdq:
  for param_seasonal in seasonal_pdq:
    try:
      model = SARIMAX(train, order= param, seasonal_order= param_seasonal, enforce_stationarity=False, enforce_invertibility= False)
      results = model.fit(disp= False)
      if results.aic < best_aic:
        best_aic = results.aic
        best_param = (param, param_seasonal)
    except:
      continue
print('Best SARIMA params:', best_param, 'AIC:', best_aic)
best_order, best_seasonal = best_param
sarima_model = SARIMAX(train, order= best_order, seasonal_order= best_seasonal, enforce_stationarity=False, enforce_invertibility=False).fit()
print(sarima_model.summary())
sarima_forecast = sarima_model.forecast(len(test))
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(test, sarima_forecast)
print('MAPE:', mape)

Saving Sample - Superstore_clean.csv to Sample - Superstore_clean (7).csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9972 entries, 0 to 9971
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     9972 non-null   int64  
 1   Row ID         9972 non-null   int64  
 2   Order ID       9972 non-null   object 
 3   Order Date     9972 non-null   object 
 4   Ship Date      9972 non-null   object 
 5   Ship Mode      9972 non-null   object 
 6   Customer ID    9972 non-null   object 
 7   Customer Name  9972 non-null   object 
 8   Segment        9972 non-null   object 
 9   Country        9972 non-null   object 
 10  City           9972 non-null   object 
 11  State          9972 non-null   object 
 12  Postal Code    9972 non-null   int64  
 13  Region         9972 non-null   object 
 14  Product ID     9972 non-null   object 
 15  Category       9972 non-null   object 
 16  Sub-Category   9972 no